# Estimate the Filling Level of a Pumped Hydro Energy Storage

This notebook estimates the filling level of a pumped hydro energy storage (PHES) using time series of pumping and production data. This data is provided by Energy Quantified and fetched via their Python client.

Most PHES plants have insufficient data or are not closed-loop systems. Two plants in Europe have been identified with good quality in their data. These are the Kruonis plant in Lithuania and the Čierny Váh plant in Slovakia.

| Plant | LT Kruonius | SK Čierny Váh |
|---|---|---|
| Max. Capacity | 900 MW | 735.16 MW |
| Efficiency | 74 % | 73 % |
| Max. Storage | 10,800 MWh | 4,000 MWh | 

In [2]:
import sys
import os

# required to use src module from parent directory
sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''), '..')))
print(f"Appended parent directory to sys.path: {os.path.abspath(os.path.join(os.path.abspath(''), '..'))}")

from datetime import date
import seaborn as sns

from energyquantified import EnergyQuantified 
from energyquantified.time import CET

from src.filling_level import estimate_filling_level

sns.set_theme(rc={'figure.figsize':(15, 7)})

Appended parent directory to sys.path: /Users/kpelz/Developer/montel/carbon_power_storage


In [ ]:
# setup EQ's Python client
eq: EnergyQuantified = EnergyQuantified(api_key_file="../eq_api_key.txt")
eq.is_api_key_valid()

In [ ]:
begin_date = date(2020, 1, 1)
end_date = date(2024, 1, 1)

## Load Data

In [ ]:
lt_net_export_ts = eq.timeseries.load(curve="LT Exchange Final Schedule Net Export MWh/h H Actual", begin=begin_date, end=end_date)
lt_net_export_ts

In [ ]:
sk_net_export_ts = eq.timeseries.load(curve="SK Exchange Final Schedule Net Export MWh/h H Actual", begin=begin_date, end=end_date)
sk_net_export_ts

In [ ]:
df = lt_net_export_ts.to_df(single_level_header=True, name="LT Net Export")
df["SK Net Export"] = sk_net_export_ts.to_df(single_level_header=True, name="SK Net Export")["SK Net Export"]
df = df.fillna(0)
df.info()

In [ ]:
df.describe()

In [ ]:
# count positive values
positive_values = df[df > 0].count()

print(f"Out of {df.count()['LT Net Export']} hours, LT had positive net exports in {positive_values['LT Net Export']} hours ({(positive_values['LT Net Export'] * 100 / df.count()['LT Net Export']):.2f} %) and SK in {positive_values['SK Net Export']} hours ({(positive_values['SK Net Export'] * 100 / df.count()['SK Net Export']):.2f} %).")